## Project 3
This project analyzes how hate crime incidents in New York City evolved during the COVID-19 pandemic and whether these trends correlate with COVID-19 case numbers.

## Prompt
- **Dataset(s) to be used:**
NYC Open Data: NYPD Hate Crimes
Dataset containing confirmed hate crime incidents in NYC
URL: https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78/about_data

- **Analysis question:** 
How did the pattern and severity of Anti-Asian hate crimes in New York City change before and during the COVID-19 pandemic, and to what extent did violent offenses (such as assaults) increase in 2021 compared with the pre-COVID period?

- **Columns that will (likely) be used:**
 ・ complaint_year_number  
 ・ month_number  
 ・ bias_motive_description  
 ・ offense_description  
 ・ complaint_date (created)
 ・ count (aggregated)
 ・ period (created for comparison)

- **Hypothesis**: 
I hypothesize that Anti-Asian hate crimes not only increased in frequency during the COVID-19 pandemic but also shifted in nature, with a substantially higher proportion of violent offenses—particularly assaults—occurring in 2021 compared with the pre-COVID period.


In [37]:
#import data
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

url = "https://data.cityofnewyork.us/resource/bqiq-cu78.csv?$limit=50000"
df = pd.read_csv(url)

# merge year and month
df["complaint_year_number"] = df["complaint_year_number"].astype(int)
df["month_number"] = df["month_number"].astype(int)

df["complaint_date"] = pd.to_datetime(
    df["complaint_year_number"].astype(str) + "-" + df["month_number"].astype(str).str.zfill(2) + "-01"
)

# group the counts of complain date
asian =  df[df["bias_motive_description"] == "ANTI-ASIAN"]

monthly =  df.groupby("complaint_date").size().reset_index(name="count")
asian_monthly = asian.groupby("complaint_date").size().reset_index(name="count")

# plot monthly-all and asian
fig = px.line(monthly, x="complaint_date", y="count", title="Monthly Hate Crimes in NYC")
fig.show()

fig = px.line(asian_monthly, x="complaint_date", y="count", title="Monthly Asian Hate Crimes  in NYC")
fig.show()


In March 2021, there was a sharp spike of 36 Anti-Asian hate crime cases in New York City.
This surge is likely associated with rising COVID-19 infection rates and the heightened anti-Asian sentiment during the pandemic.
Next, I compare the distribution of offense types in March 2021 with those observed prior to March 2020 to identify which types of crimes became most prevalent.

In [39]:
# Subset of each period
asian_mar2021 = asian[
    (asian["complaint_year_number"] == 2021) &
    (asian["month_number"] == 3)
]

asian_pre_covid = asian[
    (asian["complaint_year_number"] < 2020) |
    (
        (asian["complaint_year_number"] == 2020) &
        (asian["month_number"] <= 3)
    )
]

# definition to check the number of offense_description 
def offense_counts_by_period(sub_df: pd.DataFrame, label: str) -> pd.DataFrame:
    out = sub_df["offense_description"].value_counts().reset_index()
    out.columns = ["offense_description", "count"]
    out["period"] = label
    return out

pre_counts = offense_counts_by_period(asian_pre_covid, "Pre-COVID (–2020 Mar)")
mar_counts = offense_counts_by_period(asian_mar2021, "March 2021")

# Bar chart
compare_df = pd.concat([pre_counts, mar_counts], ignore_index=True)

fig = px.bar(
    compare_df,
    x="offense_description",
    y="count",
    color="period",
    barmode="group",
    title="Comparison of Offense Types: Pre-COVID vs March 2021",
    labels={"offense_description": "Offense Type", "count": "Count"}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

Compared with the pre-COVID period, there is a clear increase in offenses classified as Assault 3 in March 2021.
This indicates that Anti-Asian hate crimes during the pandemic did not only rise in frequency but also became more violent.
In other words, the nature of these incidents shifted from primarily non-violent harassment to physical assaults.
These findings support my original hypothesis that the pandemic was associated with an escalation in both the intensity and severity of Anti-Asian hate crimes.